In [1]:
import matplotlib.pyplot as plt
from helpers.cm26 import DatasetCM26
from helpers.computational_tools import select_NA, select_Pacific, StateFunctions, compare, select_Equator
from helpers.ann_tools import ANN, import_ANN, minibatch, export_ANN
import numpy as np
import xarray as xr
import cftime
%load_ext autoreload
%autoreload 3

In [2]:
ds = DatasetCM26()

In [3]:
ann_Txy = import_ANN('trained_models/ANN_CM26/Txy_epoch_2000.nc')
ann_Txx_Tyy = import_ANN('trained_models/ANN_CM26/Txx_Tyy_epoch_2000.nc')

# Testing on held out data

In [4]:
predictions = {}
for factor in [2,4,6,10,20]:
    predictions[factor] = []
    
for time in range(7670,8035,5):
    testing_dataset = ds.sample_epoch(time)
    for factor in [2,4,6,10,20]:
        data = testing_dataset[factor].data
        state = testing_dataset[factor].state
        ANNx = state.ANN(ann_Txy, ann_Txx_Tyy)['ZB20u']
        ANNy = state.ANN(ann_Txy, ann_Txx_Tyy)['ZB20v']
        
        pred = data[['u', 'v', 'SGSx', 'SGSy']].compute()
        pred['ANNx'] = state.ANN(ann_Txy, ann_Txx_Tyy)['ZB20u'].compute()
        pred['ANNy'] = state.ANN(ann_Txy, ann_Txx_Tyy)['ZB20v'].compute()
        
        predictions[factor].append(pred)
        
        del testing_dataset[factor].data
    print('Time is done:', time)

for factor in [2,4,6,10,20]:
    xr.concat(predictions[factor], dim='time').to_netcdf(f'Offline-skill/CM26-Surface-held-out-5day-freq/factor-{factor}.nc')

Time is done: 7670
Time is done: 7675
Time is done: 7680
Time is done: 7685
Time is done: 7690
Time is done: 7695
Time is done: 7700
Time is done: 7705
Time is done: 7710
Time is done: 7715
Time is done: 7720
Time is done: 7725
Time is done: 7730
Time is done: 7735
Time is done: 7740
Time is done: 7745
Time is done: 7750
Time is done: 7755
Time is done: 7760
Time is done: 7765
Time is done: 7770
Time is done: 7775
Time is done: 7780
Time is done: 7785
Time is done: 7790
Time is done: 7795
Time is done: 7800
Time is done: 7805
Time is done: 7810
Time is done: 7815
Time is done: 7820
Time is done: 7825
Time is done: 7830
Time is done: 7835
Time is done: 7840
Time is done: 7845
Time is done: 7850
Time is done: 7855
Time is done: 7860
Time is done: 7865
Time is done: 7870
Time is done: 7875
Time is done: 7880
Time is done: 7885
Time is done: 7890
Time is done: 7895
Time is done: 7900
Time is done: 7905
Time is done: 7910
Time is done: 7915
Time is done: 7920
Time is done: 7925
Time is done

In [12]:
data = xr.open_dataset('Offline-skill/CM26-Surface-held-out-5day-freq/factor-4.nc')

In [13]:
1 - ((data.ANNx-data.SGSx)**2).mean() / (data.SGSx**2).mean()

<xarray.DataArray ()>
array(0.50499297)

In [ ]:
def 